# Fantasy Football - Data Wrangling

In this notebook, we integrate fixture, team, Elo, and form data to construct a dataset suitable for training our goal prediction model. Our training dataset encompasses data from the Premier League seasons 2017-2018 and 2018-2019.

In [1]:
import pandas as pd
import warnings
from functools import reduce
import itertools
import numpy as np

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1) Fixtures and Results

Importing fixture data for 2017-18 and 2018-10 seasons.

In [2]:
raw_fixture_data = (pd.read_csv('data/fixture_data.csv')
                    .query('season == "2017-2018" | season == "2018-2019"'))

In [3]:
raw_fixture_data = (raw_fixture_data
                    .assign(date=pd.to_datetime(raw_fixture_data['date'], format='%d/%m/%Y'),
                            time=pd.to_datetime(raw_fixture_data['kickoff_time']).dt.strftime('%H:%M'))
                    .sort_values(by='date')
                    .assign(game_id=raw_fixture_data.index+1))

In [4]:
raw_fixture_data.head()

,game_week,week_day,date,kickoff_time,home_team,away_team,home_xg,away_xg,score,season,game_id,time
0,1,Fri,2017-08-11,19:45,Arsenal,Leicester City,2.5,1.5,4–3,2017-2018,1,19:45
1,1,Sat,2017-08-12,12:30,Watford,Liverpool,2.1,2.6,3–3,2017-2018,2,12:30
2,1,Sat,2017-08-12,15:00,Crystal Palace,Huddersfield,1.1,1.5,0–3,2017-2018,3,15:00
3,1,Sat,2017-08-12,15:00,West Brom,Bournemouth,1.3,0.5,1–0,2017-2018,4,15:00
4,1,Sat,2017-08-12,15:00,Chelsea,Burnley,1.5,0.6,2–3,2017-2018,5,15:00


Dividing the score column into home and away goal columns.

In [5]:
raw_fixture_data[['home_goals', 'away_goals']] = raw_fixture_data['score'].str.split('–', expand=True)
raw_fixture_data.drop('score', axis=1, inplace=True)

In [6]:
raw_fixture_data.head()

,game_week,week_day,date,kickoff_time,home_team,away_team,home_xg,away_xg,season,game_id,time,home_goals,away_goals
0,1,Fri,2017-08-11,19:45,Arsenal,Leicester City,2.5,1.5,2017-2018,1,19:45,4,3
1,1,Sat,2017-08-12,12:30,Watford,Liverpool,2.1,2.6,2017-2018,2,12:30,3,3
2,1,Sat,2017-08-12,15:00,Crystal Palace,Huddersfield,1.1,1.5,2017-2018,3,15:00,0,3
3,1,Sat,2017-08-12,15:00,West Brom,Bournemouth,1.3,0.5,2017-2018,4,15:00,1,0
4,1,Sat,2017-08-12,15:00,Chelsea,Burnley,1.5,0.6,2017-2018,5,15:00,2,3


To ensure the correct sequencing of game weeks based on the actual dates of matches played, we integrate FPL game weeks that correspond to the specific dates of the matches. This allows us to align the game weeks with the actual scheduling of matches, rather than relying on the pre-season schedule, which we currently have in 'game_week' column.

Importing fpl game weeks.

In [7]:
fpl_game_weeks = pd.read_csv(r'data/fpl_game_week_data.csv')

In [8]:
fpl_game_weeks = (fpl_game_weeks
                  .assign(date=pd.to_datetime(fpl_game_weeks['date'], format='%d/%m/%Y'),
                          time=pd.to_datetime(fpl_game_weeks['time']).dt.strftime('%H:%M')))


In [9]:
fpl_game_weeks.head()

,season,date,time,fpl_game_week
0,2018-2019,2018-08-10,20:00,1
1,2018-2019,2018-08-11,12:30,1
2,2018-2019,2018-08-11,15:00,1
3,2018-2019,2018-08-11,17:30,1
4,2018-2019,2018-08-12,13:30,1


Add fpl data to fixtures.

In [10]:
match_fixtures = (pd.merge(raw_fixture_data, fpl_game_weeks, on=['date', 'time', 'season'])
                 .sort_values(by='date')
                 .reset_index(drop=True)
                 .drop('kickoff_time', axis=1)
                 .drop_duplicates(subset=['home_team', 'date']))

In [11]:
match_fixtures.head()

,game_week,week_day,date,home_team,away_team,home_xg,away_xg,season,game_id,time,home_goals,away_goals,fpl_game_week
0,1,Fri,2017-08-11,Arsenal,Leicester City,2.5,1.5,2017-2018,1,19:45,4,3,1
1,1,Sat,2017-08-12,Watford,Liverpool,2.1,2.6,2017-2018,2,12:30,3,3,1
2,1,Sat,2017-08-12,Crystal Palace,Huddersfield,1.1,1.5,2017-2018,3,15:00,0,3,1
3,1,Sat,2017-08-12,West Brom,Bournemouth,1.3,0.5,2017-2018,4,15:00,1,0,1
4,1,Sat,2017-08-12,Chelsea,Burnley,1.5,0.6,2017-2018,5,15:00,2,3,1


Checking for incomplete data. Each season has 380 games and we have 2 seasons worth of data meaning we should have 760 (380 * 2) complete rows.

In [12]:
match_fixtures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   game_week      760 non-null    int64         
 1   week_day       760 non-null    object        
 2   date           760 non-null    datetime64[ns]
 3   home_team      760 non-null    object        
 4   away_team      760 non-null    object        
 5   home_xg        760 non-null    float64       
 6   away_xg        760 non-null    float64       
 7   season         760 non-null    object        
 8   game_id        760 non-null    int64         
 9   time           760 non-null    object        
 10  home_goals     760 non-null    object        
 11  away_goals     760 non-null    object        
 12  fpl_game_week  760 non-null    int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(7)
memory usage: 77.3+ KB


Getting individual rows for each team participating in a match, rather than having a single row per match.

In [13]:
team_fixtures = match_fixtures.copy()

In [14]:
home_teams = team_fixtures.rename(columns={'home_team': 'team', 'away_team': 'opponent', 'home_xg': 'team_xg',
                                           'away_xg': 'opponent_xg', 'home_goals': 'team_goals',
                                           'away_goals': 'opponent_goals'})

away_teams = team_fixtures.rename(columns={'away_team': 'team', 'home_team': 'opponent', 'away_xg': 'team_xg',
                                           'home_xg': 'opponent_xg', 'away_goals': 'team_goals',
                                           'home_goals': 'opponent_goals'})

In [15]:
team_fixtures = pd.concat([home_teams, away_teams], axis=0, ignore_index=True)

In [16]:
team_fixtures.sort_values(by=('game_id')).reset_index(drop=True).head()

,game_week,week_day,date,team,opponent,team_xg,opponent_xg,season,game_id,time,team_goals,opponent_goals,fpl_game_week
0,1,Fri,2017-08-11,Arsenal,Leicester City,2.5,1.5,2017-2018,1,19:45,4,3,1
1,1,Fri,2017-08-11,Leicester City,Arsenal,1.5,2.5,2017-2018,1,19:45,3,4,1
2,1,Sat,2017-08-12,Liverpool,Watford,2.6,2.1,2017-2018,2,12:30,3,3,1
3,1,Sat,2017-08-12,Watford,Liverpool,2.1,2.6,2017-2018,2,12:30,3,3,1
4,1,Sat,2017-08-12,Crystal Palace,Huddersfield,1.1,1.5,2017-2018,3,15:00,0,3,1


In [17]:
team_fixtures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   game_week       1520 non-null   int64         
 1   week_day        1520 non-null   object        
 2   date            1520 non-null   datetime64[ns]
 3   team            1520 non-null   object        
 4   opponent        1520 non-null   object        
 5   team_xg         1520 non-null   float64       
 6   opponent_xg     1520 non-null   float64       
 7   season          1520 non-null   object        
 8   game_id         1520 non-null   int64         
 9   time            1520 non-null   object        
 10  team_goals      1520 non-null   object        
 11  opponent_goals  1520 non-null   object        
 12  fpl_game_week   1520 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(7)
memory usage: 154.5+ KB


We now have two data frames, one with match data and one with team data.

## 2) Elo Ratings
The method we will use to measure relative strength levels is the 
Elo rating system, initially devised by physics professor 
Arpad Elo for improving chess player ratings. This 
widely adopted system can be applied to football 
teams, assigning each team a single rating. The 
difference in ratings between two teams serves as a 
predictor of the match outcome.

Adding elo data to fixture data.

In [18]:
team_elo_data = pd.read_csv('data/team_elo_data.csv')

In [19]:
team_elo_data = (team_elo_data
                 .assign(From=pd.to_datetime(team_elo_data['From'], format='%d/%m/%Y'),
                         To=pd.to_datetime(team_elo_data['To'], format='%d/%m/%Y')))

In [20]:
team_elo_data.head()

,Rank,Club,Country,Level,Elo,From,To
0,8.0,Arsenal,ENG,1,1873.913208,2017-01-02,2017-01-03
1,8.0,Arsenal,ENG,1,1869.305542,2017-01-04,2017-01-04
2,7.0,Arsenal,ENG,1,1869.305542,2017-01-05,2017-01-14
3,7.0,Arsenal,ENG,1,1875.965332,2017-01-15,2017-01-21
4,8.0,Arsenal,ENG,1,1875.965332,2017-01-22,2017-01-22


In [21]:
match_fixture_data_with_team_elos = (match_fixtures
                                     .merge(team_elo_data[['Club', 'To', 'Elo']], right_on=['Club', 'To'], left_on=['home_team', 'date'])
                                     .merge(team_elo_data[['Club', 'To', 'Elo']], right_on=['Club', 'To'], left_on=['away_team', 'date'])
                                     .rename(columns={'Elo_x': 'team_elo', 'Elo_y': 'opponent_elo'})
                                     .drop(columns=['Club_x', 'To_x', 'Club_y', 'To_y']))

In [22]:
match_fixture_data_with_team_elos.head()

,game_week,week_day,date,home_team,away_team,home_xg,away_xg,season,game_id,time,home_goals,away_goals,fpl_game_week,team_elo,opponent_elo
0,1,Fri,2017-08-11,Arsenal,Leicester City,2.5,1.5,2017-2018,1,19:45,4,3,1,1848.286499,1716.994873
1,1,Sat,2017-08-12,Watford,Liverpool,2.1,2.6,2017-2018,2,12:30,3,3,1,1603.668091,1837.415527
2,1,Sat,2017-08-12,Crystal Palace,Huddersfield,1.1,1.5,2017-2018,3,15:00,0,3,1,1642.862427,1475.799316
3,1,Sat,2017-08-12,West Brom,Bournemouth,1.3,0.5,2017-2018,4,15:00,1,0,1,1645.131348,1653.757568
4,1,Sat,2017-08-12,Chelsea,Burnley,1.5,0.6,2017-2018,5,15:00,2,3,1,1909.399658,1628.988403


In [23]:
team_fixture_data_with_team_elos = (team_fixtures
                                    .merge(team_elo_data[['Club', 'To', 'Elo']], right_on=['Club', 'To'], left_on=['team', 'date'])
                                    .merge(team_elo_data[['Club', 'To', 'Elo']], right_on=['Club', 'To'], left_on=['opponent', 'date'])
                                    .rename(columns={'Elo_x': 'team_elo', 'Elo_y': 'opponent_elo'})
                                    .drop(columns=['Club_x', 'To_x', 'Club_y', 'To_y'])
                                    .sort_values(by=['game_id'])
                                    .astype({'team_goals': 'float', 'opponent_goals': 'float'})
                                    )

In [24]:
team_fixture_data_with_team_elos.head()

,game_week,week_day,date,team,opponent,team_xg,opponent_xg,season,game_id,time,team_goals,opponent_goals,fpl_game_week,team_elo,opponent_elo
0,1,Fri,2017-08-11,Arsenal,Leicester City,2.5,1.5,2017-2018,1,19:45,4.0,3.0,1,1848.286499,1716.994873
760,1,Fri,2017-08-11,Leicester City,Arsenal,1.5,2.5,2017-2018,1,19:45,3.0,4.0,1,1716.994873,1848.286499
761,1,Sat,2017-08-12,Liverpool,Watford,2.6,2.1,2017-2018,2,12:30,3.0,3.0,1,1837.415527,1603.668091
1,1,Sat,2017-08-12,Watford,Liverpool,2.1,2.6,2017-2018,2,12:30,3.0,3.0,1,1603.668091,1837.415527
2,1,Sat,2017-08-12,Crystal Palace,Huddersfield,1.1,1.5,2017-2018,3,15:00,0.0,3.0,1,1642.862427,1475.799316


## 3) Team Form

To generate team form for different game windows (3, 5, and 10 games), we consider both actual goals scored and conceded, as well as expected goals scored and conceded. We will assess the accuracy of using various combinations of game windows and actual/expected data when developing our models.

In [25]:
games = [3, 5, 10]
rolling = [4, 6, 11]

for g, r in zip(games, rolling):
    print(g, r)
    team_fixture_data_with_team_elos[f'team_goals_scored_xg_{g}game_form'] = (team_fixture_data_with_team_elos.groupby('team')['team_xg']
        .transform(lambda x: x.rolling(r, min_periods=r)
        .sum()
        .sub(team_fixture_data_with_team_elos.team_xg) / g))
    team_fixture_data_with_team_elos[f'team_goals_conceded_xg_{g}game_form'] = (team_fixture_data_with_team_elos.groupby('team')['opponent_xg']
        .transform(lambda x: x.rolling(r, min_periods=r)
        .sum()
        .sub(team_fixture_data_with_team_elos.opponent_xg) / g)) 
    team_fixture_data_with_team_elos[f'opponent_goals_scored_xg_{g}game_form'] = (team_fixture_data_with_team_elos.groupby('opponent')['opponent_xg']
        .transform(lambda x: x.rolling(r, min_periods=r)
        .sum()
        .sub(team_fixture_data_with_team_elos.opponent_xg) / g)) 
    team_fixture_data_with_team_elos[f'opponent_goals_conceded_xg_{g}game_form'] = (team_fixture_data_with_team_elos.groupby('opponent')['team_xg']
        .transform(lambda x: x.rolling(r, min_periods=r)
        .sum()
        .sub(team_fixture_data_with_team_elos.team_xg) / g)) 
    
    
    team_fixture_data_with_team_elos[f'team_goals_scored_actual_{g}game_form'] = (team_fixture_data_with_team_elos.groupby('team')['team_goals']
        .transform(lambda x: x.rolling(r, min_periods=r)
        .sum()
        .sub(team_fixture_data_with_team_elos.team_goals) / g)) 
    team_fixture_data_with_team_elos[f'team_goals_conceded_actual_{g}game_form'] = (team_fixture_data_with_team_elos.groupby('team')['opponent_goals']
        .transform(lambda x: x.rolling(r, min_periods=r)
        .sum()
        .sub(team_fixture_data_with_team_elos.opponent_goals) / g)) 
    team_fixture_data_with_team_elos[f'opponent_goals_scored_actual_{g}game_form'] = (team_fixture_data_with_team_elos.groupby('opponent')['opponent_goals']
        .transform(lambda x: x.rolling(r, min_periods=r)
        .sum()
        .sub(team_fixture_data_with_team_elos.opponent_goals) / g)) 
    team_fixture_data_with_team_elos[f'opponent_goals_conceded_actual_{g}game_form'] = (team_fixture_data_with_team_elos.groupby('opponent')['team_goals']
        .transform(lambda x: x.rolling(r, min_periods=r)
        .sum()
        .sub(team_fixture_data_with_team_elos.team_goals) / g)) 

3 4
5 6
10 11


In [26]:
team_fixture_data_with_team_elos.tail()

,game_week,week_day,date,team,opponent,team_xg,opponent_xg,season,game_id,time,team_goals,opponent_goals,fpl_game_week,team_elo,opponent_elo,team_goals_scored_xg_3game_form,team_goals_conceded_xg_3game_form,opponent_goals_scored_xg_3game_form,opponent_goals_conceded_xg_3game_form,team_goals_scored_actual_3game_form,team_goals_conceded_actual_3game_form,opponent_goals_scored_actual_3game_form,opponent_goals_conceded_actual_3game_form,team_goals_scored_xg_5game_form,team_goals_conceded_xg_5game_form,opponent_goals_scored_xg_5game_form,opponent_goals_conceded_xg_5game_form,team_goals_scored_actual_5game_form,team_goals_conceded_actual_5game_form,opponent_goals_scored_actual_5game_form,opponent_goals_conceded_actual_5game_form,team_goals_scored_xg_10game_form,team_goals_conceded_xg_10game_form,opponent_goals_scored_xg_10game_form,opponent_goals_conceded_xg_10game_form,team_goals_scored_actual_10game_form,team_goals_conceded_actual_10game_form,opponent_goals_scored_actual_10game_form,opponent_goals_conceded_actual_10game_form
758,38,Sun,2019-05-12,Liverpool,Wolves,1.6,1.3,2018-2019,758,15:00,2.0,0.0,38,2041.960815,1725.302734,2.433333,0.900000,1.733333,0.700000,3.333333,0.666667,2.000000,0.666667,2.08,0.86,1.74,1.04,3.0,0.6,1.4,1.0,2.10,0.71,1.49,1.08,2.8,0.7,1.2,1.0
750,38,Sun,2019-05-12,Manchester Utd,Cardiff City,1.5,1.6,2018-2019,759,15:00,0.0,2.0,38,1864.029297,1589.286865,1.133333,1.066667,1.366667,1.700000,0.666667,1.333333,0.666667,2.000000,1.24,1.24,1.18,1.66,0.8,1.8,0.8,1.6,1.51,1.33,1.09,1.74,1.3,1.7,0.7,1.7
1510,38,Sun,2019-05-12,Cardiff City,Manchester Utd,1.6,1.5,2018-2019,759,15:00,2.0,0.0,38,1589.286865,1864.029297,1.366667,1.700000,1.133333,1.066667,0.666667,2.000000,0.666667,1.333333,1.18,1.66,1.24,1.24,0.8,1.6,0.8,1.8,1.09,1.74,1.51,1.33,0.7,1.7,1.3,1.7
759,38,Sun,2019-05-12,Watford,West Ham,2.2,3.1,2018-2019,760,15:00,1.0,4.0,38,1705.565552,1717.171753,1.266667,2.100000,1.400000,1.000000,0.666667,2.000000,2.000000,0.666667,1.14,1.92,1.24,1.48,0.8,1.6,1.4,1.2,0.97,1.90,1.07,1.83,1.2,2.0,1.3,1.4
1519,38,Sun,2019-05-12,West Ham,Watford,3.1,2.2,2018-2019,760,15:00,4.0,1.0,38,1717.171753,1705.565552,1.400000,1.000000,1.266667,2.100000,2.000000,0.666667,0.666667,2.000000,1.24,1.48,1.14,1.92,1.4,1.2,0.8,1.6,1.07,1.83,0.97,1.90,1.3,1.4,1.2,2.0


Adding elo difference between two teams.

In [27]:
team_fixture_data_with_team_elos['elodiff'] = team_fixture_data_with_team_elos['team_elo'] - team_fixture_data_with_team_elos['opponent_elo']

In [28]:
team_fixture_data_with_team_elos[['fpl_game_week', 'team', 'opponent', 'elodiff']].reset_index(drop=True).head()

,fpl_game_week,team,opponent,elodiff
0,1,Arsenal,Leicester City,131.291626
1,1,Leicester City,Arsenal,-131.291626
2,1,Liverpool,Watford,233.747436
3,1,Watford,Liverpool,-233.747436
4,1,Crystal Palace,Huddersfield,167.063111


Adding column showing if team is playing at home or away.

In [29]:
home_away_data = match_fixture_data_with_team_elos[['game_id', 'home_team']]

In [30]:
team_fixture_data_with_team_elos = pd.merge(team_fixture_data_with_team_elos, home_away_data, on='game_id')

In [31]:
team_fixture_data_with_team_elos[['fpl_game_week', 'date', 'team', 'opponent', 'home_team']].head()

,fpl_game_week,date,team,opponent,home_team
0,1,2017-08-11,Arsenal,Leicester City,Arsenal
1,1,2017-08-11,Leicester City,Arsenal,Arsenal
2,1,2017-08-12,Liverpool,Watford,Watford
3,1,2017-08-12,Watford,Liverpool,Watford
4,1,2017-08-12,Crystal Palace,Huddersfield,Crystal Palace


In [32]:
team_fixture_data_with_team_elos = (team_fixture_data_with_team_elos
                                   .assign(home=lambda x: (x['team'] == x['home_team']).astype(int))
                                   .drop('home_team', axis=1))

In [33]:
team_fixture_data_with_team_elos[['fpl_game_week', 'date', 'team', 'opponent', 'home']].head()

,fpl_game_week,date,team,opponent,home
0,1,2017-08-11,Arsenal,Leicester City,1
1,1,2017-08-11,Leicester City,Arsenal,0
2,1,2017-08-12,Liverpool,Watford,0
3,1,2017-08-12,Watford,Liverpool,1
4,1,2017-08-12,Crystal Palace,Huddersfield,1


Our machine learning model, does not accept NaN values, as a result we need to eliminate all such occurrences. For the chosen form metric, depending on the number of past games considered (3, 5, or 10 game weeks), the initial 3, 5, or 10 game weeks of the 2017-18 season will contain NaN values. Additionally, the first 3, 5, or 10 game weeks for matches involving recently promoted teams will also yield NaN values. These NaN values will be replaced by the 25th and 75th percentiles of the league's goals scored and conceded, as promoted clubs often exhibit comparatively poor performance levels.

In [34]:
team_fixture_data_with_team_elos = team_fixture_data_with_team_elos.query('game_id>50')

In [35]:
team_fixture_data_with_team_elos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1420 entries, 100 to 1519
Data columns (total 41 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   game_week                                   1420 non-null   int64         
 1   week_day                                    1420 non-null   object        
 2   date                                        1420 non-null   datetime64[ns]
 3   team                                        1420 non-null   object        
 4   opponent                                    1420 non-null   object        
 5   team_xg                                     1420 non-null   float64       
 6   opponent_xg                                 1420 non-null   float64       
 7   season                                      1420 non-null   object        
 8   game_id                                     1420 non-null   int64         
 9   time       

Filling promoted team na values with the 25th percentile of goals scored and 75th percentile of goals conceded by teams in league.

In [36]:
team_fixture_data_with_team_elos.describe()

,game_week,date,team_xg,opponent_xg,game_id,team_goals,opponent_goals,fpl_game_week,team_elo,opponent_elo,team_goals_scored_xg_3game_form,team_goals_conceded_xg_3game_form,opponent_goals_scored_xg_3game_form,opponent_goals_conceded_xg_3game_form,team_goals_scored_actual_3game_form,team_goals_conceded_actual_3game_form,opponent_goals_scored_actual_3game_form,opponent_goals_conceded_actual_3game_form,team_goals_scored_xg_5game_form,team_goals_conceded_xg_5game_form,opponent_goals_scored_xg_5game_form,opponent_goals_conceded_xg_5game_form,team_goals_scored_actual_5game_form,team_goals_conceded_actual_5game_form,opponent_goals_scored_actual_5game_form,opponent_goals_conceded_actual_5game_form,team_goals_scored_xg_10game_form,team_goals_conceded_xg_10game_form,opponent_goals_scored_xg_10game_form,opponent_goals_conceded_xg_10game_form,team_goals_scored_actual_10game_form,team_goals_conceded_actual_10game_form,opponent_goals_scored_actual_10game_form,opponent_goals_conceded_actual_10game_form,elodiff,home
count,1420.000000,1420,1420.000000,1420.000000,1420.000000,1420.000000,1420.000000,1420.000000,1420.000000,1420.000000,1411.000000,1411.000000,1411.000000,1411.000000,1411.000000,1411.000000,1411.000000,1411.000000,1405.000000,1405.000000,1405.000000,1405.000000,1405.000000,1405.000000,1405.000000,1405.000000,1290.000000,1290.000000,1290.000000,1290.000000,1290.000000,1290.000000,1290.000000,1290.000000,1420.000000,1420.000000
mean,20.661972,2018-07-21 07:32:16.901408512,1.299155,1.299155,405.500000,1.385915,1.385915,20.733803,1723.898285,1723.898285,1.292157,1.288826,1.292157,1.288826,1.380109,1.375384,1.380109,1.375384,1.292071,1.286776,1.292071,1.286776,1.380214,1.375089,1.380214,1.375089,1.295341,1.282581,1.295341,1.282581,1.391628,1.375891,1.391628,1.375891,0.000000,0.500000
min,1.000000,2017-09-23 00:00:00,0.000000,0.000000,51.000000,0.000000,0.000000,1.000000,1499.452393,1499.452393,0.266667,0.200000,0.266667,0.200000,0.000000,0.000000,0.000000,0.000000,0.320000,0.240000,0.320000,0.240000,0.000000,0.000000,0.000000,0.000000,0.540000,0.370000,0.540000,0.370000,0.300000,0.200000,0.300000,0.200000,-511.554199,0.000000
25%,12.000000,2018-01-14 00:00:00,0.700000,0.700000,228.000000,0.000000,0.000000,12.000000,1635.075440,1635.075440,0.900000,0.933333,0.900000,0.933333,0.666667,0.666667,0.666667,0.666667,0.940000,1.020000,0.940000,1.020000,0.800000,1.000000,0.800000,1.000000,1.000000,1.090000,1.000000,1.090000,0.900000,1.000000,0.900000,1.000000,-118.505676,0.000000
50%,21.000000,2018-08-25 00:00:00,1.200000,1.200000,405.500000,1.000000,1.000000,21.000000,1677.043762,1677.043762,1.200000,1.266667,1.200000,1.266667,1.333333,1.333333,1.333333,1.333333,1.240000,1.280000,1.240000,1.280000,1.200000,1.400000,1.200000,1.400000,1.240000,1.320000,1.240000,1.320000,1.300000,1.400000,1.300000,1.400000,0.000000,0.500000
75%,30.000000,2019-01-01 00:00:00,1.700000,1.700000,583.000000,2.000000,2.000000,30.000000,1851.385345,1851.385345,1.616667,1.600000,1.616667,1.600000,2.000000,2.000000,2.000000,2.000000,1.540000,1.540000,1.540000,1.540000,1.800000,1.800000,1.800000,1.800000,1.510000,1.500000,1.510000,1.500000,1.700000,1.700000,1.700000,1.700000,118.505676,1.000000
max,38.000000,2019-05-12 00:00:00,4.800000,4.800000,760.000000,7.000000,7.000000,38.000000,2041.960815,2041.960815,3.500000,3.033333,3.500000,3.033333,5.333333,4.333333,5.333333,4.333333,3.120000,2.680000,3.120000,2.680000,4.800000,3.400000,4.800000,3.400000,2.660000,2.130000,2.660000,2.130000,3.700000,2.800000,3.700000,2.800000,511.554199,1.000000
std,10.398601,NaN,0.779674,0.779674,205.031349,1.260479,1.260479,10.485380,127.995431,127.995431,0.519896,0.479868,0.519896,0.479868,0.795586,0.775489,0.795586,0.775489,0.457885,0.398609,0.457885,0.398609,0.689342,0.618776,0.689342,0.618776,0.400474,0.324059,0.400474,0.324059,0.589927,0.476736,0.589927,0.476736,186.913790,0.500176


In [37]:
games

[3, 5, 10]

In [38]:
for g in games:
    team_fixture_data_with_team_elos[f'team_goals_scored_xg_{g}game_form'].fillna(team_fixture_data_with_team_elos[f'team_goals_scored_xg_{g}game_form'].quantile(.25), inplace=True)
    team_fixture_data_with_team_elos[f'opponent_goals_scored_xg_{g}game_form'].fillna(team_fixture_data_with_team_elos[f'team_goals_scored_xg_{g}game_form'].quantile(.75), inplace=True)
    team_fixture_data_with_team_elos[f'team_goals_conceded_xg_{g}game_form'].fillna(team_fixture_data_with_team_elos[f'team_goals_scored_xg_{g}game_form'].quantile(.75), inplace=True)
    team_fixture_data_with_team_elos[f'opponent_goals_conceded_xg_{g}game_form'].fillna((team_fixture_data_with_team_elos[f'team_goals_scored_xg_{g}game_form'].quantile(.25)), inplace=True)
    
    team_fixture_data_with_team_elos[f'team_goals_scored_actual_{g}game_form'].fillna(team_fixture_data_with_team_elos[f'team_goals_scored_actual_{g}game_form'].quantile(.25), inplace=True)
    team_fixture_data_with_team_elos[f'opponent_goals_scored_actual_{g}game_form'].fillna(team_fixture_data_with_team_elos[f'team_goals_scored_actual_{g}game_form'].quantile(.75), inplace=True)
    team_fixture_data_with_team_elos[f'team_goals_conceded_actual_{g}game_form'].fillna(team_fixture_data_with_team_elos[f'team_goals_scored_actual_{g}game_form'].quantile(.75), inplace=True)
    team_fixture_data_with_team_elos[f'opponent_goals_conceded_actual_{g}game_form'].fillna(team_fixture_data_with_team_elos[f'team_goals_scored_actual_{g}game_form'].quantile(.25), inplace=True)

Checking that there are no more NaN values.

In [39]:
team_fixture_data_with_team_elos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1420 entries, 100 to 1519
Data columns (total 41 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   game_week                                   1420 non-null   int64         
 1   week_day                                    1420 non-null   object        
 2   date                                        1420 non-null   datetime64[ns]
 3   team                                        1420 non-null   object        
 4   opponent                                    1420 non-null   object        
 5   team_xg                                     1420 non-null   float64       
 6   opponent_xg                                 1420 non-null   float64       
 7   season                                      1420 non-null   object        
 8   game_id                                     1420 non-null   int64         
 9   time       

Amount of games we have data for by team and season.

In [40]:
team_fixture_data_with_team_elos.groupby(['team', 'season'])['game_id'].count()

team             season   
Arsenal          2017-2018    33
                 2018-2019    38
Bournemouth      2017-2018    33
                 2018-2019    38
Brighton         2017-2018    33
                 2018-2019    38
Burnley          2017-2018    33
                 2018-2019    38
Cardiff City     2018-2019    38
Chelsea          2017-2018    33
                 2018-2019    38
Crystal Palace   2017-2018    33
                 2018-2019    38
Everton          2017-2018    33
                 2018-2019    38
Fulham           2018-2019    38
Huddersfield     2017-2018    33
                 2018-2019    38
Leicester City   2017-2018    33
                 2018-2019    38
Liverpool        2017-2018    33
                 2018-2019    38
Manchester City  2017-2018    33
                 2018-2019    38
Manchester Utd   2017-2018    33
                 2018-2019    38
Newcastle Utd    2017-2018    33
                 2018-2019    38
Southampton      2017-2018    33
                

Exporting our final df to use for model training.

In [41]:
team_fixture_data_with_team_elos.to_csv('wrangled_data_final.csv', index=False)